# NLP Visualization Tests

This notebook tests the visualization utilities for NLP as provided by `trulens.vis.nlp.NLP`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

# ! pip uninstall -y trulens

# Use this if running this notebook from within its place in the truera repository.
sys.path.insert(0, "..")

import pandas as pd
import numpy as np
import random
from dataclasses import dataclass
from pathlib import Path
import torch
from torch import Tensor
from torch import nn
import re
from typing import Iterable, List

from trulens.nn.models import get_model_wrapper
from trulens.vis.nlp import NLP

In [ ]:
from typing import Generic, TypeVar

Token = str
Word = str
Part = TypeVar("Part", Token, Word)

max_tokens = 8

# Collections of synonyms for test data generation for toy sentiment models.
GOOD_SYNONYMS = ["good", "alright", "buena", "well", "nice", "decent", "best"]
BAD_SYNONYMS = [
    "bad", "mala", "naughty", "rotten", "amiss", "wicked", "negative",
    "unfavourable", "horrid"
]
NEUTRAL_SYNONYMS = [
    "neutral"
]  #, "neutrality"] # not including neutrality as the greedy tokenizer cannot handle it due to it having a prefix that is another token.
SYNONYMS = dict(good=GOOD_SYNONYMS, bad=BAD_SYNONYMS, neutral=NEUTRAL_SYNONYMS)

@dataclass
class Span(Generic[Part]):
    """
    Tokens or words along with indices into the string from which they were
    derived.
    """

    item: Part
    begin: int
    end: int

class GreedyTokenizer:
    # The following special tokens mimic use of huggingface tokenizers like Bert.

    # The following tokens must be unique.

    # Whitespace/separator.
    space_token = "[SPC]"

    # Input separator, used at end of an token input sequence
    sep_token = "[SEP]"

    # Unknown tokens, everything that is not a vocabulary word mapped to unknown.
    unk_token = "[UNK]"

    # Padding to input token sequences up to a fixed length.
    pad_token = "[PAD]"

    # Begining of input token. Token sequences start with this.
    cls_token = "[CLS]"

    # Regex for various separator characters including whitespace and punctuation.
    r_whitespace = r"(?P<whitespace>[\s\.\!\;\:])"

    # Regex for non-separator characters.
    r_blackspace = r"(?P<blackspace>\S)"

    def __init__(self, max_tokens: int = 8):
        self.mask_token = '[MASK]'
        self.sep_token = GreedyTokenizer.sep_token
        self.cls_token = GreedyTokenizer.cls_token

        # parent defines accessors:
        self.pad_token = GreedyTokenizer.pad_token
        self.unk_token = GreedyTokenizer.unk_token

        self.vocab = {
            self.unk_token: 0,
            self.pad_token: 1,
            self.mask_token: 2,
            self.sep_token: 3,
            self.cls_token: 4,
            "neutral": 5,
            "good": 6,
            "bad": 7
        }

        # Also add synonyms that tokenize to the same tokens as the original words.
        for token, synonyms in SYNONYMS.items():
            for syn in synonyms:
                self.vocab[syn] = self.vocab[token]

        self.ids = {i: k for k, i in self.vocab.items()}

        self.pad_token_id = self.vocab[self.pad_token]
        self.cls_token_id = self.vocab[self.cls_token]
        self.sep_token_id = self.vocab[self.sep_token]
        self.unk_token_id = self.vocab[self.unk_token]

        self.normal_tokens = [tok for tok in self.vocab.keys() if tok[0] != "["]

        # Regex for all normal (non-special) tokens.
        self.r_toks = "(?P<token>" + (
            "|".join(re.escape(tok) for tok in self.normal_tokens)
        ) + ")"

        self.max_tokens = max_tokens

        # Regex pattern for finding a token, a whitespace, or a non-whitespace.
        # It is important that non-whitespace comes after tokens as they are
        # potentially made of the same characters, with token having precedence.
        self.pattern = re.compile(
            (
                "|".join(
                    [
                        self.r_toks, GreedyTokenizer.r_whitespace,
                        GreedyTokenizer.r_blackspace
                    ]
                )
            )
        )

    def decode(self, token_id: int) -> str:
        return self.ids[token_id]

    def _greedy_tokenize(self, text: str) -> List[Span]:
        """
        Tokenize a text string into token spans in a greedy manner.
        """
        spans = []

        # Accumulators for constructing separator and unknown tokens. Regexp
        # reads whitespace and "blackspace" characters, one at a time, if no
        # token is found. These need to be accumulated into one large spacing or
        # unknown token.
        current_type = None

        # For every match of main pattern.
        for m in self.pattern.finditer(text.lower()):
            # Type of match (token/whitespace/blackspace)
            type = m.lastgroup
            # Matching string.
            tok = m.groupdict()[type]
            # Its span in input text.
            span = m.span(type)

            # Replace whitespace and blackspace matches with temporary special
            # token indicators.
            if type == "token":
                pass
            elif type == "whitespace":
                tok = self.space_token
            elif type == "blackspace":
                tok = self.unk_token

            # Accumulte separators and unknown tokens.
            if type != "token":
                if current_type == type:
                    spans[-1].end = span[1]
                else:
                    current_type = type
                    spans.append(
                        Span(tok, begin=span[0], end=span[1])
                    )

            # Otherwise append a non-special token.
            else:
                spans.append(Span(tok, begin=span[0], end=span[1]))
                current_type = None

        return spans

    def _tokenize(self, text: str) -> dict:
        all_spans = self._greedy_tokenize(text)

        spans = []
        input_ids = []

        def add_special(tok):
            spans.append(Span(item=tok, begin=0, end=0))
            input_ids.append(self.vocab[tok])

        add_special(self.sep_token)

        for span in all_spans:
            if span.item in [self.space_token]:
                pass
            else:
                spans.append(span)
                input_ids.append(self.vocab[span.item])

            if len(spans) + 1 >= self.max_tokens:
                break

        add_special(self.cls_token)

        while len(spans) < self.max_tokens:
            add_special(self.pad_token)

        return dict(
            input_ids=np.array(input_ids)
        )

    def tokenize(self, texts: Iterable[str]) -> dict:
        toks = [self._tokenize(t) for t in texts]
        ins = []
        for tok in toks:
            ins.append(tok['input_ids'])

        return dict(input_ids=torch.tensor(np.array(ins)))
    
tokenizer = GreedyTokenizer(max_tokens=max_tokens)

In [ ]:
tokenizer.tokenize(np.array(["hello there"]))

In [ ]:
def generate_dataset(num_rows: int, row_length: int, seed=0xdeadbeef):
    """
    Generate random sentiment sentences and their labels. Uses only the tokens
    "good", "bad", "neutral", and their synonyms but also creates words that
    combine these tokens. Output class is determined by which of the tokens
    appears most frequently. Can be tokenized by GreedyTokenizer. 
    """

    random.seed(a=seed)

    ret = []
    cls = []
    for i in range(num_rows):
        sent = []

        goods = 0
        bads = 0

        while len(sent) < row_length * 2:
            r = random.random()

            word = "neutral"

            if r > 0.9:
                word = "good"
                goods += 1
            elif r > 0.8:
                word = "bad"
                bads += 1
            elif r > 0.2:
                word = " "

            if word != " ":
                word = random.choice(SYNONYMS[word])

            sent.append(word)

        ret.append("".join(sent))

        if goods > bads:
            gt = 0
        elif bads > goods:
            gt = 1
        else:
            gt = random.randint(0, 1)

        cls.append(gt)

    return pd.DataFrame(dict(sentence=ret, sentiment=cls))

In [ ]:
dataset = generate_dataset(1000, row_length=max_tokens-1)
sentences = dataset['sentence'].to_numpy()
sentiments = dataset['sentiment'].to_numpy()

In [ ]:
@dataclass
class Outputs:
    logits: Tensor = None
    probits: Tensor = None


class SentimentSoft(torch.nn.Module):

    def set_parameters(self) -> None:
        """Set model parameters as per fixed specification."""

        Wi = torch.zeros_like(
            self.lstm.weight_ih_l0
        )  # order: W_ii|W_if|W_ig|W_io
        bi = torch.zeros_like(self.lstm.bias_ih_l0)  # order b_ii|b_if|b_ig|b_io
        Wh = torch.zeros_like(
            self.lstm.weight_hh_l0
        )  # order W_hi|W_hf|W_hg|W_ho
        bh = torch.zeros_like(self.lstm.bias_hh_l0)  # b_hi|b_hf|b_hg|b_ho

        big = 2.0  # Multipliers to help dealing with LSTM sigmoids.
        # half = 4.0  # Intention here is that sigmoid((x*big) - half) is ~0 if x is ~0; and
        # ~1 when x is >~ 1.

        # internal states
        S_POSITIVITY = 0

        # words/tokens
        W_UNKNOWN = tokenizer.unk_token_id
        W_NEUTRAL = tokenizer.vocab['neutral']
        W_GOOD = tokenizer.vocab['good']
        W_BAD = tokenizer.vocab['bad']

        hs = self.hidden_size

        # make sure c gate is always big, so tanh(c) is always ~1 .
        bi[0:hs * 3] = big * 10.0
        bh[0:hs * 3] = big * 10.0

        # o gate weights:
        Wi[3 * hs + S_POSITIVITY, W_NEUTRAL] = 0  # ignore neutral word
        Wi[3 * hs + S_POSITIVITY, W_GOOD] = big  # read good word
        Wi[3 * hs + S_POSITIVITY, W_BAD] = -big  # read bad word
        Wh[3 * hs + S_POSITIVITY, S_POSITIVITY] = big  #
        bh[3 * hs + S_POSITIVITY] = -0.5 * big

        self.lstm.weight_hh_l0 = nn.Parameter(Wh)
        self.lstm.bias_hh_l0 = nn.Parameter(bh)
        self.lstm.weight_ih_l0 = nn.Parameter(Wi)
        self.lstm.bias_ih_l0 = nn.Parameter(bi)

        self.embedding.weight = nn.Parameter(torch.eye(self.emb_size))

        self.logits.weight = nn.Parameter(
            torch.tensor([
                [1.0],  # positive
                [-1.0],  # negative
            ])
        )
        self.logits.bias = nn.Parameter(
            torch.tensor([
                0.0,  # positive
                1.0,  # negative
            ])
        )

    def __init__(self):
        super().__init__()

        # self.model_path = model_path

        # self.tokenizer = CustomTokenizerWrapper(model_path=self.model_path)

        self.labels = ['+', '-']

        self.emb_size = 8

        self.hidden_size = 1

        # Identity embedding, each vocab word has its own dimension where its presence is encoded.
        self.embedding = nn.Embedding(
            padding_idx=1,
            embedding_dim=self.emb_size,
            num_embeddings=self.emb_size
        )

        # here only to get us a separate layer for the lstm's first input (the embeddings)
        self.lstm_embedding_input = nn.Identity()

        self.lstm = nn.LSTM(
            input_size=self.emb_size,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True
        )

        # Linear layer to combine the two types of confused state and weight things so that
        # confused outweighs positive and negative, while positive and negative outweigh neutral
        # if more than one of these states is set.
        self.logits = torch.nn.Linear(
            in_features=self.hidden_size, out_features=2, bias=True
        )

        self.logits_squeezed = nn.Identity()

        # Finally add a softmax for classification.
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(
        self,
        input_ids: torch.Tensor = None,
        *,
        inputs_embeds: torch.Tensor = None,
        attention_mask: torch.Tensor = None,
        token_type_ids: torch.Tensor = None
    ) -> torch.Tensor:
        # Signature and some functionality imitiates huggingface models.
        # TODO: Use `attention_mask`.

        if input_ids is not None:
            batch_size = input_ids.shape[0]
            device = input_ids.device
        else:
            batch_size = inputs_embeds.shape[0]
            device = inputs_embeds.device

        S_POSITIVITY = 0

        h0 = torch.zeros(1, batch_size, self.hidden_size).to(device)
        h0[:, :, S_POSITIVITY] = 0.5  # initial state is neutral
        c0 = 2.0 * torch.ones(1, batch_size, self.hidden_size).to(device)

        if inputs_embeds is None:
            embeds = self.embedding(input_ids)
        else:
            embeds = inputs_embeds

        # here only to get us a separate layer for the lstm's first input (the embeddings)
        embeds = self.lstm_embedding_input(embeds)

        _, (hn, _) = self.lstm(embeds, (h0, c0))

        logits = self.logits(hn)[0]

        logits_squeezed = self.logits_squeezed(logits)

        probits = self.softmax(logits_squeezed)

        # outputs also imitates hugging face
        return Outputs(logits=logits, probits=probits)


model = SentimentSoft()
model.set_parameters()

In [ ]:
scores = model(**tokenizer.tokenize(sentences)).logits.detach().numpy()
preds = np.argmax(scores, axis=1)
acc = (preds == sentiments).mean()
print(f"accuracy={100.0*acc:0.3f}%")

wrongs = np.argwhere(preds != sentiments)
for t, s, p, gt in zip(sentences[wrongs], scores[wrongs], preds[wrongs], sentiments[wrongs]):
    print (model.labels[p[0]], s, model.labels[gt[0]], t, tokenizer.tokenize(t))

In [ ]:
wrapper = get_model_wrapper(model, input_shape=(None, tokenizer.max_tokens), device="cpu")

In [ ]:
wrapper.print_layer_names()

In [ ]:
texts=['good', "bad", "nothing", "good and bad"]

In [ ]:
# Minimal usage provides only tokenize and input_accessor if neccessary.

V = NLP(
    tokenize=tokenizer.tokenize,
    input_accessor=lambda x: x['input_ids'],
)
V.tokens(texts=texts)

In [ ]:
# Test hidden tokens

V = NLP(
    tokenize=tokenizer.tokenize,
    input_accessor=lambda x: x['input_ids'],
    hidden_tokens=set([tokenizer.pad_token_id])
    # do not display these tokens
)
V.tokens(texts=texts)

In [ ]:
# Test decode to show readable representations of token id's.

V = NLP(
    decode=tokenizer.decode,
    tokenize=tokenizer.tokenize,
    input_accessor=lambda x: x['input_ids'],
    hidden_tokens=set([tokenizer.pad_token_id])
)
V.tokens(texts=texts)

In [ ]:
# Show token id's alongside readable forms.

V.tokens(texts=texts, show_id=True)

In [ ]:
# Also show pre-tokenized text.

V.tokens(texts=texts, show_id=True, show_text=True)

In [ ]:
# Test model outputs.

V = NLP(
    wrapper=wrapper,
    # labels=model.labels,
    decode=tokenizer.decode,
    tokenize=tokenizer.tokenize,
    # huggingface models can take as input the keyword args as per produced by their tokenizers.

    input_accessor=lambda x: x['input_ids'],
    # for huggingface models, input/token ids are under input_ids key in the input dictionary

    output_accessor=lambda x: x.logits,
    # and logits under 'logits' key in the output dictionary

    hidden_tokens=set([tokenizer.pad_token_id])
    # do not display these tokens
)
V.tokens(texts=texts)

In [ ]:
# Test model outputs with labels.

V = NLP(
    wrapper=wrapper,
    labels=model.labels,
    decode=lambda x: tokenizer.decode(x),
    tokenize=lambda sentences: tokenizer.tokenize(sentences),
    # huggingface models can take as input the keyword args as per produced by their tokenizers.

    input_accessor=lambda x: x['input_ids'],
    # for huggingface models, input/token ids are under input_ids key in the input dictionary

    output_accessor=lambda x: x.logits,
    # and logits under 'logits' key in the output dictionary

    hidden_tokens=set([tokenizer.pad_token_id])
    # do not display these tokens
)
V.tokens(texts=texts)

In [ ]:
# Test attributions; various QoI, point DoI.

from trulens.nn.distributions import PointDoi, GaussianDoi, LinearDoi
from trulens.nn.attribution import InternalInfluence, IntegratedGradients, Cut, OutputCut
from trulens.nn.quantities import MaxClassQoI, ClassQoI, ComparativeQoI

common_args = dict(
    doi=PointDoi(Cut('embedding')),
    model=wrapper,
    cuts=(Cut('embedding'), OutputCut(accessor=lambda o: o.logits))
)

attributors = [
    InternalInfluence(qoi=MaxClassQoI(), **common_args),
    InternalInfluence(qoi=ClassQoI(1), **common_args),
    InternalInfluence(qoi=ClassQoI(0), **common_args),
    InternalInfluence(qoi=ComparativeQoI(0, 1), **common_args)
]

for infl in attributors:

    V = NLP(
        wrapper=wrapper,
        labels=model.labels,
        decode=lambda x: tokenizer.decode(x),
        tokenize=lambda sentences: tokenizer.tokenize(sentences),
        # huggingface models can take as input the keyword args as per produced by their tokenizers.
        input_accessor=lambda x: x['input_ids'],
        # for huggingface models, input/token ids are under input_ids key in the input dictionary
        output_accessor=lambda x: x.logits,
        # and logits under 'logits' key in the output dictionary
        hidden_tokens=set([tokenizer.pad_token_id])
        # do not display these tokens
    )

    display(V.tokens(texts=texts, attributor=infl))

In [ ]:
common_args = dict(qoi=MaxClassQoI(), model=wrapper, cuts=(Cut('embedding'), OutputCut(accessor=lambda o: o.logits)))

attributors = [
    InternalInfluence(
        doi=PointDoi(Cut('embedding')),
        **common_args
    ),
    InternalInfluence(
        doi=GaussianDoi(var=0.1, resolution=10, cut=Cut('embedding')),
        **common_args
    ),
    InternalInfluence(
        doi=LinearDoi(resolution=10, cut=Cut('embedding')),
        **common_args
    ),
]

for infl in attributors:

    V = NLP(
        wrapper=wrapper,
        labels=model.labels,
        decode=lambda x: tokenizer.decode(x),
        tokenize=lambda sentences: tokenizer.tokenize(sentences),
        # huggingface models can take as input the keyword args as per produced by their tokenizers.

        input_accessor=lambda x: x['input_ids'],
        # for huggingface models, input/token ids are under input_ids key in the input dictionary

        output_accessor=lambda x: x.logits,
        # and logits under 'logits' key in the output dictionary

        hidden_tokens=set([tokenizer.pad_token_id])
        # do not display these tokens
    )

    display(V.tokens(texts=texts, attributor=infl))

In [ ]:
# Test stability pairs.

V = NLP(
    wrapper=wrapper,
    labels=model.labels,
    decode=lambda x: tokenizer.decode(x),
    tokenize=lambda sentences: tokenizer.tokenize(sentences),
    # huggingface models can take as input the keyword args as per produced by their tokenizers.

    input_accessor=lambda x: x['input_ids'],
    # for huggingface models, input/token ids are under input_ids key in the input dictionary

    output_accessor=lambda x: x.logits,
    # and logits under 'logits' key in the output dictionary

    hidden_tokens=set([tokenizer.pad_token_id])
    # do not display these tokens
)

texts1 = ['this is good', "good good bad good"]
texts2 = ['this is bad', "good bad bad bad"]

V.tokens_stability(texts1=texts1, texts2=texts2)

In [ ]:
# Test doi enumeration.

common_args = dict(return_grads=True, return_doi=True, qoi=MaxClassQoI(), model=wrapper, cuts=(Cut('embedding'), OutputCut(accessor=lambda o: o.logits)))

attributors = [
    InternalInfluence(
        doi=PointDoi(Cut('embedding')),
        **common_args
    ),
    InternalInfluence(
        doi=GaussianDoi(var=0.4, resolution=20, cut=Cut('embedding')),
        **common_args
    ),
    InternalInfluence(
        doi=LinearDoi(resolution=20, cut=Cut('embedding')),
        **common_args
    ),
]

for infl in attributors:

    V = NLP(
        wrapper=wrapper,
        labels=model.labels,
        decode=lambda x: tokenizer.decode(x),
        tokenize=lambda sentences: tokenizer.tokenize(sentences),
        embedder=model.embedding,
        embeddings=list(model.embedding.parameters())[0].detach().numpy(),
        # huggingface models can take as input the keyword args as per produced by their tokenizers.

        input_accessor=lambda x: x['input_ids'],
        # for huggingface models, input/token ids are under input_ids key in the input dictionary

        output_accessor=lambda x: x.logits,
        # and logits under 'logits' key in the output dictionary

        hidden_tokens=set([tokenizer.pad_token_id])
        # do not display these tokens
    )

    display(f"doi={infl.doi}")
    display(V.tokens(texts=texts, attributor=infl, show_doi=True))

In [ ]:
# Test doi enumeration with token stability pairs.

common_args = dict(
    return_grads=True, 
    return_doi=True, 
    qoi=MaxClassQoI(), 
    model=wrapper, 
    cuts=(Cut('embedding'), OutputCut(accessor=lambda o: o.logits))
)

attributors = [
    InternalInfluence(
        doi=PointDoi(Cut('embedding')),
        **common_args
    ),
    InternalInfluence(
        doi=GaussianDoi(var=0.1, resolution=10, cut=Cut('embedding')),
        **common_args
    ),
    InternalInfluence(
        doi=LinearDoi(resolution=10, cut=Cut('embedding')),
        **common_args
    ),
]

for infl in attributors:

    V = NLP(
        wrapper=wrapper,
        labels=model.labels,
        decode=lambda x: tokenizer.decode(x),
        tokenize=lambda sentences: tokenizer.tokenize(sentences),
        embedder=model.embedding,
        embeddings=list(model.embedding.parameters())[0].detach().numpy(),
        # huggingface models can take as input the keyword args as per produced by their tokenizers.

        input_accessor=lambda x: x['input_ids'],
        # for huggingface models, input/token ids are under input_ids key in the input dictionary

        output_accessor=lambda x: x.logits,
        # and logits under 'logits' key in the output dictionary

        hidden_tokens=set([tokenizer.pad_token_id])
        # do not display these tokens
    )

    display(V.tokens_stability(texts1=texts1, texts2=texts2, attributor=infl, show_doi=True))

In [ ]:
# Test IG baselines.

from trulens.utils.nlp import token_baseline_swap
from trulens.utils.typing import ModelInputs

inputs_swap_baseline_ids, inputs_swap_baseline_embeddings = token_baseline_swap(
    
    token_pairs = [(
        tokenizer.vocab['good'],
        tokenizer.vocab['bad']
    )],

    input_accessor=lambda x: x.kwargs['input_ids'],

    ids_to_embeddings=model.embedding
    # Callable to produce embeddings from token ids.
)

from trulens.utils.nlp import token_baseline

inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
    keep_tokens=set([tokenizer.cls_token_id, tokenizer.sep_token_id]),
    # Which tokens to preserve.

    replacement_token=tokenizer.pad_token_id,
    # What to replace tokens with.

    input_accessor=lambda x: x.kwargs['input_ids'],
    # input_accessor = lambda x: x, 

    ids_to_embeddings=model.embedding
    # Callable to produce embeddings from token ids.
)

common_args = dict(
    return_grads=True, 
    return_doi=True, 
    qoi=ClassQoI(1), 
    model=wrapper, 
    cuts=(Cut('embedding'), OutputCut(accessor=lambda o: o.logits))
)

attributors = [InternalInfluence(
        doi=LinearDoi(resolution=20, cut=Cut('embedding'), baseline=baseline),
        **common_args
    ) for baseline in [
        None,
        inputs_baseline_embeddings,
        inputs_swap_baseline_embeddings
    ]]

for infl in attributors:

    V = NLP(
        wrapper=wrapper,
        labels=model.labels,
        decode=lambda x: tokenizer.decode(x),
        tokenize=lambda sentences: tokenizer.tokenize(sentences),
        embedder=model.embedding,
        embeddings=list(model.embedding.parameters())[0].detach().numpy(),
        # huggingface models can take as input the keyword args as per produced by their tokenizers.

        input_accessor=lambda x: x['input_ids'],
        # for huggingface models, input/token ids are under input_ids key in the input dictionary

        output_accessor=lambda x: x.logits,
        # and logits under 'logits' key in the output dictionary

        hidden_tokens=set([tokenizer.pad_token_id])
        # do not display these tokens
    )
    display(f"baseline={infl.doi.baseline}")

    display(V.tokens(texts=texts, attributor=infl, show_doi=True))

In [ ]:
# check different embedding distance methods for finding closest tokens for interventions

common_args = dict(
    return_grads=True, 
    return_doi=True, 
    qoi=ClassQoI(1), 
    model=wrapper, 
    cuts=(Cut('embedding'), OutputCut(accessor=lambda o: o.logits))
)

attributor = InternalInfluence(
        doi=LinearDoi(resolution=20, cut=Cut('embedding'), baseline=inputs_baseline_embeddings),
        **common_args
    )

for dist in ['l1', 'l2', 'cosine']:

    V = NLP(
        wrapper=wrapper,
        labels=model.labels,
        decode=lambda x: tokenizer.decode(x),
        tokenize=lambda sentences: tokenizer.tokenize(sentences),
        embedder=model.embedding,
        embeddings=list(model.embedding.parameters())[0].detach().numpy(),
        embedding_distance=dist,
        # huggingface models can take as input the keyword args as per produced by their tokenizers.

        input_accessor=lambda x: x['input_ids'],
        # for huggingface models, input/token ids are under input_ids key in the input dictionary

        output_accessor=lambda x: x.logits,
        # and logits under 'logits' key in the output dictionary

        hidden_tokens=set([tokenizer.pad_token_id])
        # do not display these tokens
    )
    display(f"distance={dist}")

    display(V.tokens(texts=["good and bad"], attributor=attributor, show_doi=True))